In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

import shutil
from train import run
import os
import itertools
import glob

def write_errormsg2file(msg, error_file_name):
    if not os.path.isfile(error_file_name):
        with open(error_file_name, 'w') as f:
            f.write(f'error : {msg}\n')
    else:
        with open(error_file_name, 'a') as f:
            f.write(f'error : {msg}\n')
            
def do_exps(exps_dict= None, general_opts= None, device= None, exp_dir = '../figs/test', save_special= False):    
    exp_idx = 0
    keys= list(exps.keys())
    
    val_list_list= []
    key_list = [] #eg: 'MODEL.MODEL_A.rotation_lambda'
    key_suffix_list= [] # eg: 'rotation_lambda'
    
    for key, val_list in exps_dict.items():
        key_list.append(key)
        key_suffix_list.append(key.split('.')[-1])
        
        val_list_list.append(val_list)
        
    attr_combination_list = [list(s) for s in itertools.product(*val_list_list)]
    
    print(f'number of total experiments : {len(attr_combination_list)}')
    
    count_already_trained=0
    count_train_from_begining=0
    for attr_combination in attr_combination_list:
        save_dir = f'{exp_dir}/'
        opts= []
        for idx in range(len(attr_combination)):
            opts += [key_list[idx], attr_combination[idx]]
            attr= attr_combination[idx]
            
            #####
            attr_is_list= False
            try:attr_is_list = isinstance(eval(attr), list)
            except:pass
            if attr_is_list:
                attr= '_'.join(list(map(str, eval(attr)))) ## [, ] should not be in the directory name because glob is sensitive to that !
            #####
            
            save_dir+= f'{key_suffix_list[idx]}({attr})@'
    
        save_dir = save_dir[:-1] # remove last '@' 

        exp_idx+=1
        opts_other= ['NAME', f'exp_idx({exp_idx})', 
                     'GENERAL.device', device, 
                     'GENERAL.save_dir', save_dir
                    ]
                     
        opts_other+= general_opts

        opts = opts_other + opts

        
        if len(glob.glob(f'{save_dir}/1_*.jpg'))!=0:
            count_already_trained+=1
            print(f'PASSING :: {save_dir}')
            continue
        
        count_train_from_begining+=1
                    
        try:shutil.rmtree(save_dir)
        except:pass

        save_folder_name= save_dir.split('/')[-1]
        if len(save_folder_name)>255:
            print(f'\nFolder length is too long: len(results_saving_folder) -> {len(save_folder_name)} (<= 255)')
            print(save_folder_name)
        
        #run(opts= opts, save_special=save_special)
        try:
            run(opts= opts, save_special=save_special)
        except:
            error_file_name = f'{exp_dir}/errors.txt'
            write_errormsg2file(f'ERROR : {save_dir}', error_file_name)
            print(f'ERROR : {save_dir}')
    print('count_already_trained : ', count_already_trained)
    print('count_train_from_begining : ', count_train_from_begining)

In [3]:
exp_dir= '../figs/mnistv23'
#!rm -rf $exp_dir
#!mkdir $exp_dir

In [ ]:
device = 'cuda:0'

exps = {
    'DATASET.img_size': ['512'], #'128', '256' -> create a dataset
    'DATASET.num_samples_train': ['600', '3000', '6000'], #'20000'
    'MODEL.MODEL_DECODER.upsample_net': ['custom_v2']
}

general_opts= ['TRAIN.show_results_epoch', '5',
               'TRAIN.epochs', '150',

               'MODEL.MODEL_DECODER.upsample_net_init_method', 'Ht_based', #'Ht_based OR xavier_normal'
               'MODEL.MODEL_DECODER.custom_upsampling_bias', 'True', #keep bias True
               
               'MODEL.MODEL_H.lr_H', '1.0',
               'MODEL.MODEL_A.lambda_scale_factor', '4',
               'MODEL.MODEL_A.rotation_lambda', '10000.0',
               'MODEL.MODEL_H.T', '8',
              ]

do_exps(exps, general_opts, device, exp_dir = exp_dir, save_special= False)

In [ ]:
'''
device = 'cuda:0'

exps = {
    'MODEL.MODEL_DECODER.custom_upsampling_bias': ['True', 'False'],
    'DATASET.num_samples_train': ['60'], #'20000' 
    'MODEL.MODEL_DECODER.upsample_net_init_method': ['Ht_based'], #'xavier_normal'
    'MODEL.MODEL_DECODER.upsample_net': ['custom_v2', 'custom_v5', 'custom_v6', 'custom_v7'], 
    'MODEL.MODEL_H.lr_H': ['1.0']} #'1.0', '5.0'

general_opts= ['TRAIN.show_results_epoch', '1',
               'TRAIN.epochs', '2',
               
               'MODEL.MODEL_A.lambda_scale_factor', '4',
               'MODEL.MODEL_A.rotation_lambda', '10000.0',
               'MODEL.MODEL_H.T', '8',
               'DATASET.img_size', '64',
              ]

do_exps(exps, general_opts, device, exp_dir = exp_dir, save_special= False)
'''